## Preprocess

In [ ]:
from data_processing.preprocess_encoder_data import filter_and_dedup_encoder_data

filter_and_dedup_encoder_data(
    encoder_csv_path='data_processing/raw/encoder_data.csv',
    stroke_map_ty_path='data_processing/mapping/stroke_mapping_name.csv',
    hit_area_map_path='data_processing/mapping/hit_area_mapping.csv',
    stroke_map_llm_path='data_processing/mapping/stroke_mapping_llm.csv',
    output_csv_path='data_processing/processed/filtered_encoder_data.csv',
    dedup_keep='last',       
    sort_by=None             
)

## VideoCropper 

VideoCropper 是用來從完整比賽視頻中裁切出特定動作片段的工具，主要功能：

### 核心功能
- 根據 frame 編號從完整視頻中裁切出 16 幀的短片段
- 每個動作前後各保留 7-8 幀，確保動作完整性
- 保持原始視頻的 FPS 和解析度

### 初始化參數
- **output_path**: 裁切後視頻的輸出目錄
- **video_dir**: 原始完整視頻的存放目錄（預設 'Dataset/Video'）

### crop_videos 方法
根據 CSV 數據批量裁切視頻：

- **csv_path**: 包含 id 欄位的 CSV 檔案路徑
- **裁切邏輯**: 
  - 從 id 中解析出 game 和 frame_num
  - 裁切範圍：frame_num-7 到 frame_num+8（共16幀）
  - 輸出檔名：{id}.mp4

### 使用流程
1. 確保原始視頻檔案存在於 video_dir 中
2. 初始化 VideoCropper 並指定輸出路徑
3. 調用 crop_videos() 傳入 CSV 路徑
4. 自動生成所有動作的短視頻片段

### 檔案命名範例
- 輸入：game33_set3_128437 → 輸出：game33_set3_128437.mp4
- 從 game33.mp4 裁切第 128430-128445 幀

In [ ]:
from dataset_pipeline.video_cropper import VideoCropper

cropper = VideoCropper(output_path='Output',video_dir='Dataset/Video')
cropper.crop_videos('data_processing/processed/filtered_encoder_data.csv')

## CaptionGenerator 

CaptionGenerator 是用來為羽球動作生成描述文字的工具，主要功能：

### 核心功能
- 為每個 stroke 動作生成自然語言描述
- 結合 player、stroke_name、hit_area 等信息生成完整句子
- 輸出包含 image_id 和 caption 的 DataFrame

### generate_captions 方法
根據 CSV 數據為每個動作生成描述：

- **csv_path**: 處理後的數據檔案路徑（包含 id, player, stroke_name, hit_area 等欄位）
- **返回值**: pandas DataFrame，包含 image_id 和對應的 caption

### 使用流程
1. 初始化 CaptionGenerator（指定輸出目錄）
2. 調用 generate_captions() 傳入 CSV 路徑
3. 將結果保存為 CSV 檔案，供 QAGenerator 使用

In [ ]:
import os
from dataset_pipeline.caption_generator import CaptionGenerator
csv_path = 'data_processing/processed/filtered_encoder_data.csv'
caption_output_dir = 'generated_labels/caption'
os.makedirs(caption_output_dir, exist_ok=True)

caption_gen = CaptionGenerator(caption_output_dir)
caption_df = caption_gen.generate_captions(csv_path)
caption_csv = os.path.join(caption_output_dir, 'dataset_labels_caption.csv')
caption_df.to_csv(caption_csv, index=False)
print(f'Captions saved to: {caption_csv}')


## QAGenerator

QAGenerator 是用來生成問答數據集的工具，主要功能：

### 核心參數
- **templates_with_all**: 包含 player, stroke, hit_area 的問題模板
- **templates_player_stroke**: 包含 player, stroke 的問題模板  
- **templates_stroke_only**: 只包含 stroke 的問題模板
- **templates_hit_area_only**: 只包含 hit_area 的問題模板
- **stroke_chunk_size**: 每個 chunk 包含的 stroke 數量（預設5）
- **caption_csv_path**: caption 檔案路徑

### generate_by_rally 方法
將數據按 rally 分組，每個 rally 再分成固定大小的 chunk，為每個 chunk 生成問答對：

- **csv_path**: 處理後的數據檔案路徑
- **output_path**: 輸出的 QA 數據集路徑
- **num_questions_per_rally**: 每個 chunk 生成的問題數量（包含負樣本和正樣本）
- **val_to_train_ratio**: 驗證集移到訓練集的比例
- **use_cot**: 是否使用 Chain-of-Thought 格式的答案

### 問題生成邏輯
1. **負樣本**: 生成 4 種類型的負樣本問題（答案為"The event does not occur"）
2. **正樣本**: 從 chunk 中隨機採樣生成正樣本問題（答案指向具體的 stroke 位置）
3. **模板選擇**: 隨機從對應類型的模板中選擇問題格式

In [ ]:
import os
from dataset_pipeline.qa_generator import QAGenerator
csv_path = 'data_processing/processed/filtered_encoder_data.csv'
caption_csv = 'generated_labels/caption/dataset_labels_caption.csv'
qa_output_dir = 'generated_labels/QA_chunk_5'
os.makedirs(qa_output_dir, exist_ok=True)
qa_output_path = os.path.join(qa_output_dir, 'qa_dataset.json')

templates_all = [
    "When does the {player} hits a {stroke} {hit_area}?",
    "At what point in the rally does the {player} perform a {stroke} {hit_area}?",
    "Identify when the {player} executes a {stroke} {hit_area}.",
    "Locate the moment when a {stroke} {hit_area} is played by the {player}."
]

templates_ps = [
    "When does the {player} hits a {stroke}?",
    "At what time does the {player} execute a {stroke}?",
    "When in this rally does the {player} perform a {stroke}?",
    "Identify the moment the {player} plays a {stroke}."
]

templates_s = [
    "When is a {stroke} hits?",
    "At what moment does a {stroke} occur?",
    "When during the rally does a {stroke} happen?",
    "Locate when the {stroke} takes place."
]

templates_h = [
    "Which stroke is hit {hit_area}?",
    "Which shot lands {hit_area}?",
    "Identify the stroke that occurs {hit_area}.",
    "What stroke happens {hit_area}?"
]
qa_gen = QAGenerator(
    templates_with_all=templates_all,
    templates_player_stroke=templates_ps,
    templates_stroke_only=templates_s,
    templates_hit_area_only=templates_h,
    stroke_chunk_size=5,
    caption_csv_path=caption_csv
)
qa_gen.generate_by_rally(
    csv_path=csv_path,
    output_path=qa_output_path,
    num_questions_per_rally=14,
    val_to_train_ratio=0.05,
    use_cot=False
)
print(f'QA dataset saved to: {qa_output_path}')
